# Stage 01: Setup 
## Step 01: Initialize

This step initializes the workflow and validates the environment.

**Purpose:**
- Verify database connectivity
- Validate cycle configuration
- Set up working directories
- Initialize logging

In [1]:
from system import WorkContext, Step, db, display
# Create work context - automatically detects cycle/stage/step from notebook path
context = WorkContext()
print(f"{context}")

ModuleNotFoundError: No module named 'system'

In [ ]:
# Create step tracker
step = Step(
    context=context,
    idempotent=False,  # This step cannot be re-run once completed
    auto_start=True    # Automatically start the run
)

# Check if we can execute
if not step.can_execute:
    display.print_error("Cannot execute step")
    print(step.status_message)
    raise Exception("Step cannot be executed")

display.print_success("Step can execute")

In [ ]:
try:
    display.print_header("Initializing Workflow")
    
    # Log progress
    step.log("Starting initialization")
    
    # 1. Test database connection
    print("\n1. Testing database connection...")
    if db.test_connection():
        display.print_success("Database connected")
        step.log("Database connection verified")
    else:
        raise Exception("Database connection failed")
    
    # 2. Verify active cycle
    print("\n2. Verifying active cycle...")
    active = db.get_active_cycle()
    if active:
        display.print_success(f"Active cycle: {active['cycle_name']}")
        step.log(f"Active cycle verified: {active['cycle_name']}")
    else:
        raise Exception("No active cycle found")
    
    # 3. Set up working directories
    print("\n3. Setting up directories...")
    from pathlib import Path
    
    cycle_dir = Path(f"home/jovyan/workspace/workflows/Active_{active['cycle_name']}")
    files_dir = cycle_dir / "files"
    logs_dir = cycle_dir / "logs"
    
    files_dir.mkdir(exist_ok=True)
    logs_dir.mkdir(exist_ok=True)
    
    display.print_success(f"Files directory: {files_dir}")
    display.print_success(f"Logs directory: {logs_dir}")
    step.log("Directories created")
    
    # Save checkpoint
    step.checkpoint({
        'cycle_dir': str(cycle_dir),
        'files_dir': str(files_dir),
        'logs_dir': str(logs_dir)
    })
    
    # 4. Initialize logging
    print("\n4. Initializing logging...")
    import logging
    from datetime import datetime
    
    log_file = logs_dir / f"initialization_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
    logging.basicConfig(
        filename=str(log_file),
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s'
    )
    logging.info("Workflow initialized")
    
    display.print_success(f"Log file: {log_file}")
    step.log(f"Logging initialized: {log_file}")
    
    # Prepare output data
    output = {
        'cycle_name': active['cycle_name'],
        'cycle_id': active['id'],
        'files_dir': str(files_dir),
        'logs_dir': str(logs_dir),
        'log_file': str(log_file),
        'initialization_complete': True
    }
    
    # Mark step as completed
    step.complete(output)
    
    display.print_header("INITIALIZATION COMPLETE")
    
except Exception as e:
    # Mark step as failed
    step.fail(str(e))
    display.print_error(f"Step failed: {e}")
    raise

In [ ]:
display.print_header("Step Status")

# Get step history
from system import cycle_manager

history = cycle_manager.get_step_history(
    context.cycle_name,
    context.stage_num,
    context.step_num
)

if not history.empty:
    # Format duration
    history['duration'] = history['duration_seconds'].apply(display.format_duration)
    
    # Display recent runs
    display_df = history[[
        'run_number', 'status', 'started_ts', 'completed_ts', 'duration'
    ]].head(5)
    
    display.display_dataframe(display_df, title="Step Run History")
else:
    display.print_info("No run history available")

# Show last output
last_output = step.get_last_output()
if last_output:
    display.print_subheader("Last Output")
    display.display_json(last_output, indent=2)